In [12]:
import sys

In [20]:
import shutil
import os
import pandas as pd

# import own modules
sys.path.append("..")  # Adds higher directory to python modules path.
# from scripts import features as ft
from scripts import preprocessing as pp



In [4]:
# path to csv file
path_df = os.path.join("..", "data", "df.csv")
# recalculate ???
recalculate_df = False
# get features
if os.path.isfile(path_df) and not recalculate_df:
    df = pd.read_csv(path_df, index_col=0)
else:
    df = ft.get_features()
    df.to_csv(path_df)
# set id as index
df = df.set_index("id", drop=True)

In [5]:
df.head()

,asd,img,sp_idx,sp_fix_count,sp_fix_duration_ms_total,sp_fix_duration_ms_mean,sp_fix_duration_ms_var,sp_len_px_total,sp_saccade_amplitude_px_mean,sp_saccade_amplitude_px_var,...,sal_first_above_0.75*max_rank,sal_first_above_0.9*max_rank,sal_mean,sal_sum,sal_max,sal_weighted_duration_sum,sal_weighted_duration_mean,sal_KLD,sal_NSS,dummy_feature_name
id,,,,,,,,,,,,,,,,,,,,,
asd_126_00,1,126,0,12,2098,174.833333,17110.305556,1734.793605,157.708510,6535.662390,...,20,20,25.000000,300.0,114.0,50501.0,4208.416667,4.442845,0.415974,12
asd_126_01,1,126,1,12,3572,297.666667,7305.555556,1772.309555,161.119050,7533.378852,...,20,20,46.583333,559.0,147.0,158865.0,13238.750000,3.000474,1.136514,12
asd_126_02,1,126,2,5,349,69.800000,2691.360000,837.418654,209.354663,13255.374874,...,20,20,48.200000,241.0,156.0,22773.0,4554.600000,3.890714,1.190484,5
asd_126_03,1,126,3,1,50,50.000000,0.000000,0.000000,0.000000,0.000000,...,20,20,0.000000,0.0,0.0,0.0,0.000000,26.467882,-0.418627,1
asd_126_04,1,126,4,1,108,108.000000,0.000000,0.000000,0.000000,0.000000,...,20,20,111.000000,111.0,111.0,11988.0,11988.000000,1.895202,3.287003,1


In [17]:
# train-test-split
X = df.drop({"img", "sp_idx", "dummy_feature_name"}, axis=1)
y = X.pop("asd")
X_train, X_test, y_train, y_test = pp.split(X, y)
print(f"test-set has '{len(y_test)}' samples - out of '{df.shape[0]}'")
print(f"  ~ {len(y_test) / df.shape[0] * 100:.2f}% of full dataset")

test-set has '500' samples - out of '7598'
  ~ 6.58% of full dataset


In [22]:
X_train_df = pd.DataFrame(X_train)
X_test_df = pd.DataFrame(X_test)

In [24]:

def copy_images_by_id(df, source_folder, destination_folder):
    """
    Copies images from Individual_FixMaps folder to Train folder based on the id (index) in the X_train_df dataFrame.
    Args:
    - df : Dataframe where the functions copy images by id will be called for
    - source_folder : folder where image will be copied from
    - destination_folder : folder where images will be copied to
    Returns:
    - None
    """
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    asd = []
    filename = []
    # Loop through the index of the X_train_df
    for id in df.index:
        source_path = os.path.join(source_folder, f"{id}.jpg")
        destination_path = os.path.join(destination_folder, f"{id}.jpg")
        filename.append(destination_path)
        if 'asd' in id:
            asd.append('1')
        else:
            asd.append('0')
        # Check if the image file exists in the "Individual_FixMaps" folder
        if os.path.exists(source_path):
        # Copy the image file to the "Train" folder
            shutil.copyfile(source_path, destination_path)
            print(f"Copied image {id}.jpg")
        else:
            print(f"Image {id}.jpg not found in the source folder.")
    dataframe_obtained = pd.DataFrame({'asd': asd, 'filename' : filename})
    return dataframe_obtained
# define source folder
source_folder = '/Users/stefanschloegl/Desktop/neuefische/Capstone_project/data/Saliency4ASD/TrainingData/Individual_FixMaps'
# define destination folder
destination_folder = '/Users/stefanschloegl/Desktop/neuefische/Capstone_project/data/CNN/Train'
train_data = copy_images_by_id(X_train_df, source_folder, destination_folder)
train_data.to_csv(os.path.join(destination_folder, "..", 'train_data.csv'), index=False)


Copied image asd_126_00.jpg
Copied image asd_126_01.jpg
Copied image asd_126_02.jpg
Copied image asd_126_03.jpg
Copied image asd_126_04.jpg
Copied image asd_126_05.jpg
Copied image asd_126_06.jpg
Copied image asd_126_07.jpg
Copied image asd_126_08.jpg
Copied image asd_126_09.jpg
Copied image asd_126_10.jpg
Copied image asd_126_11.jpg
Copied image asd_126_12.jpg
Copied image asd_132_00.jpg
Copied image asd_132_01.jpg
Copied image asd_132_02.jpg
Copied image asd_132_03.jpg
Copied image asd_132_04.jpg
Copied image asd_132_05.jpg
Copied image asd_132_06.jpg
Copied image asd_132_07.jpg
Copied image asd_132_08.jpg
Copied image asd_132_09.jpg
Copied image asd_132_10.jpg
Copied image asd_132_11.jpg
Copied image asd_132_12.jpg
Copied image asd_077_00.jpg
Copied image asd_077_01.jpg
Copied image asd_077_02.jpg
Copied image asd_077_03.jpg
Copied image asd_077_04.jpg
Copied image asd_077_05.jpg
Copied image asd_077_06.jpg
Copied image asd_077_07.jpg
Copied image asd_077_08.jpg
Copied image asd_077

In [25]:
# define destination folder for test
destination_folder = '/Users/stefanschloegl/Desktop/neuefische/Capstone_project/data//CNN/Test'
test_data = copy_images_by_id(X_test_df, source_folder, destination_folder)
test_data.to_csv(os.path.join(destination_folder, "..", 'test_data.csv'), index=False)

Copied image asd_287_00.jpg
Copied image asd_287_01.jpg
Copied image asd_287_02.jpg
Copied image asd_287_03.jpg
Copied image asd_287_04.jpg
Copied image asd_287_05.jpg
Copied image asd_287_06.jpg
Copied image asd_287_07.jpg
Copied image asd_287_08.jpg
Copied image asd_287_09.jpg
Copied image asd_287_10.jpg
Copied image asd_287_11.jpg
Copied image asd_287_12.jpg
Copied image asd_253_00.jpg
Copied image asd_253_01.jpg
Copied image asd_253_02.jpg
Copied image asd_253_03.jpg
Copied image asd_253_04.jpg
Copied image asd_253_05.jpg
Copied image asd_253_06.jpg
Copied image asd_253_07.jpg
Copied image asd_253_08.jpg
Copied image asd_253_09.jpg
Copied image asd_253_10.jpg
Copied image asd_253_11.jpg
Copied image asd_120_00.jpg
Copied image asd_120_01.jpg
Copied image asd_120_02.jpg
Copied image asd_120_03.jpg
Copied image asd_120_04.jpg
Copied image asd_120_05.jpg
Copied image asd_120_06.jpg
Copied image asd_120_07.jpg
Copied image asd_120_08.jpg
Copied image asd_120_09.jpg
Copied image asd_120